In [6]:
%cd ..

c:\Users\hp zbook\Documents\GitHub\Geasture_ESP


In [7]:
import pandas as pd

# Load your Excel file
data = pd.read_excel(r"data\sensor_data.xlsx")

print(data)

      Timestamp  Accel X  Accel Y  Accel Z  Gyro X  Gyro Y  Gyro Z     lable
0      14:49:24     2.36    -8.24    -3.47   -0.01    0.30   -0.16      down
1      14:49:24     3.21    -9.22    -2.42    0.11   -0.41    0.22      down
2      14:49:24     1.83   -10.16    -2.82    0.13    0.00   -0.02      down
3      14:49:24     2.58    -9.41    -2.58   -0.04    0.32   -0.16      down
4      14:49:24     3.34    -8.63    -2.44   -0.17    0.20   -0.05      down
...         ...      ...      ...      ...     ...     ...     ...       ...
13728  19:11:07    -0.71    -0.69     8.65    0.00   -0.04    0.01  striaght
13729  19:11:07    -0.69    -0.73     8.68    0.02   -0.01    0.01  striaght
13730  19:11:08    -0.69    -0.69     8.63    0.00   -0.05    0.01  striaght
13731  19:11:08    -0.60    -0.71     8.66    0.00   -0.03    0.01  striaght
13732  19:11:08    -0.84    -0.67     8.61    0.01    0.01    0.01  striaght

[13733 rows x 8 columns]


In [8]:
# Clean column names
data.columns = data.columns.str.strip().str.lower().str.replace(" ", "_")

# Drop missing values
data_cleaned = data.dropna()

# Define numerical features
features = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

# IQR outlier removal
def remove_outliers(df, features):
    Q1 = df[features].quantile(0.25)
    Q3 = df[features].quantile(0.75)
    IQR = Q3 - Q1
    mask = ~((df[features] < (Q1 - 1.5 * IQR)) | (df[features] > (Q3 + 1.5 * IQR))).any(axis=1)
    return df[mask]


# Apply for each movement
filtered_data = pd.concat([
    remove_outliers(data_cleaned[data_cleaned['lable'] == label], features)
    for label in data_cleaned['lable'].unique()
])
for label in filtered_data['lable'].unique():
    subset = filtered_data[filtered_data['lable'] == label]


In [9]:
from sklearn.preprocessing import StandardScaler

# Extract features and labels
features = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
X_raw = filtered_data[features]
y_raw = filtered_data['lable']

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

# Recreate the DataFrame with labels
normalized_data = pd.DataFrame(X_scaled, columns=features)
normalized_data['label'] = y_raw.values

In [11]:
# Apply function
X_seq, y_seq = create_sequences(normalized_data, features)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_seq)
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
# Save the normalized data to a new CSV file
normalized_data.to_csv(r"data\preprocessed_sensor_data.csv", index=False)
print("Preprocessed data saved to 'preprocessed_sensor_data.csv'")


Preprocessed data saved to 'preprocessed_sensor_data.csv'


In [20]:
import joblib

joblib.dump(scaler, "models/scaler.pkl")

['models/scaler.pkl']